In [ ]:
from __future__ import print_function
%matplotlib inline
import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
DIR='bench_py_v012'
data = {}
empty_files = []
for fname in sorted(os.listdir(DIR)):
    if fname.startswith('run'):
        ffname = DIR+'/'+fname
        with open(ffname) as f:
            for l in f.readlines():
                if l[0]=='#' and 'num_memes' in l:
                    #print("Loading", fname)
                    d = dict(e.split('=') for e in l[1:].strip().split(', '))
                    df = pd.read_csv(ffname, comment='#', sep=' ')
                    if len(df) > 0:
                        key = (d['mutator_class'],d['temperature'],d['mutation_scale'])
                        data[key] = (df.iloc[-1].Comprehension, df.iloc[-1].MeanGrammarDrift)
                    else:
                        empty_files.append(fname)
                    break
            
if len(empty_files):
    print("Following files were empty:")
    for f in empty_files:
        print("- ", f)

In [ ]:
def floatsorted(l):
    return sorted(l, key=lambda x: float(x))

mutator_values, temp_values, scale_values = map(list, map(set, zip(*data.keys())))
mutator_values = sorted(mutator_values)
temp_values    = floatsorted(temp_values)
scale_values   = floatsorted(scale_values)

ms_ixs = { v:k for k, v in enumerate(scale_values) }
temp_ixs = { v:k for k, v in enumerate(temp_values) }


In [ ]:
plt.figure(figsize=(10,50))
gs = gridspec.GridSpec(len(mutator_values), 2)
for ndxm, mutator in enumerate(mutator_values):
    for ndx, (col, vmax) in enumerate([('final_value',1.0), ('final_value_grammardrift', 0.0025)]):
        plt.subplot(gs[ndxm, ndx])
        mx = np.zeros((len(ms_ixs), len(temp_ixs)))
        for k, v in data.items():
            if k[0] == mutator:
                mx[ms_ixs[k[2]],temp_ixs[k[1]]] = v[ndx]
        i = plt.imshow(mx, aspect='auto', vmin=0, vmax=vmax, interpolation='nearest', origin='lower')
        plt.xticks(range(len(temp_values)), temp_values, rotation=45)
        plt.yticks(range(len(scale_values)), scale_values)
        plt.gca().set_yticklabels(scale_values)
        if ndx  == 0: plt.ylabel('Mutation scale')
        if ndxm == len(mutator_values)-1: plt.xlabel('Temperature')
        plt.colorbar(ticks=[0, vmax])
        if ndx == 0:
            plt.title(mutator+'\nComprehension')
        else:
            plt.title(mutator+'\nDrift')
            
plt.tight_layout()
plt.savefig('comprehension_and_drift.pdf')